In [ ]:
# %% [markdown]
# # Canadian Architectural & Engineering Services Price Index
# **Interactive Dashboard Preparation**  
# *Last Updated: {current_date}*

# %% [markdown]
# ## 1. Setup and Data Loading

# %%
import pandas as pd
import geopandas as gpd
import plotly.express as px
from plotly.io import write_html
from pathlib import Path
import numpy as np

# Create directories
Path("docs/plots").mkdir(parents=True, exist_ok=True)
Path("data/processed").mkdir(parents=True, exist_ok=True)

# %%
# Load and clean data
aespi_raw = pd.read_csv("data/raw/18100164.csv")

# Clean columns
aespi = aespi_raw.rename(columns={
    "REF_DATE": "Date",
    "GEO": "Region",
    "North American Industry Classification System (NAICS)": "Service Category",
    "VALUE": "Index Value"
}).drop(columns=["DGUID", "UOM", "UOM_ID", "SCALAR_FACTOR", "SCALAR_ID", 
                 "VECTOR", "COORDINATE", "STATUS", "SYMBOL", "TERMINATED", "DECIMALS"])

# Convert and filter dates
aespi["Date"] = pd.to_datetime(aespi["Date"])
aespi = aespi[aespi["Date"] >= "2018-01-01"]

# Categorical ordering
region_order = ['Canada', 'Atlantic Region', 'Quebec', 'Ontario', 'Prairie Region', 'British Columbia and territories']
service_order = [
    "Architectural, engineering and related services price index",
    "Architectural and landscape architectural services",
    "Engineering services",
    "Surveying and mapping services"
]
aespi["Region"] = pd.Categorical(aespi["Region"], categories=region_order, ordered=True)
aespi["Service Category"] = pd.Categorical(aespi["Service Category"], categories=service_order, ordered=True)

# %% [markdown]
# ## 2. Geographic Data Processing

# %%
# Load and process map data
provinces = gpd.read_file("https://naturalearth.s3.amazonaws.com/10m_cultural/ne_10m_admin_1_states_provinces.zip")
canada = provinces[provinces['admin'] == 'Canada'].copy()

region_mapping = {
    'Atlantic Region': ['New Brunswick', 'Nova Scotia', 'Prince Edward Island', 'Newfoundland and Labrador'],
    'Quebec': ['Québec'],
    'Ontario': ['Ontario'],
    'Prairie Region': ['Manitoba', 'Saskatchewan', 'Alberta'],
    'British Columbia and territories': ['British Columbia', 'Yukon', 'Northwest Territories', 'Nunavut']
}

canada['region'] = canada['name'].apply(lambda x: next(k for k,v in region_mapping.items() if x in v))
regions = canada.dissolve(by='region', as_index=False)
regions['geometry'] = regions.geometry.simplify(0.01)

# %% [markdown]
# ## 3. Analysis and Visualization

# %%
# Time Series Analysis
fig_ts = px.line(aespi, x='Date', y='Index Value', 
                 color='Region',
                 facet_col='Service Category',
                 facet_col_wrap=2,
                 height=900,
                 title="Service Price Trends by Region (2018-2024)")

fig_ts.update_layout(
    hovermode="x unified",
    plot_bgcolor='white',
    margin=dict(r=100)
)

# %%
# Annual Change Calculation
aespi["Year"] = aespi["Date"].dt.year
annual_avg = aespi.groupby(["Region", "Service Category", "Year"])["Index Value"].mean().reset_index()
annual_avg["Yearly Change (%)"] = annual_avg.groupby(["Region", "Service Category"])["Index Value"].pct_change() * 100
annual_change = annual_avg.dropna(subset=["Yearly Change (%)"])

fig_change = px.bar(annual_change,
                    x='Year', y='Yearly Change (%)',
                    color='Service Category',
                    facet_col='Region',
                    facet_col_wrap=3,
                    barmode='group',
                    title="Yearly Percentage Changes by Region")

# %%
# Choropleth Map
regional_avg = aespi.groupby('Region')['Index Value'].mean().reset_index()
merged = regions.merge(regional_avg, left_on='region', right_on='Region')

fig_map = px.choropleth(merged,
                        geojson=merged.geometry,
                        locations=merged.index,
                        color='Index Value',
                        hover_name='region',
                        projection='natural earth',
                        title="Regional Price Index Averages (2018-2024)")

fig_map.update_geos(fitbounds="locations", visible=False)

# %% [markdown]
# ## 4. Export Interactive Visualizations

# %%
# Export HTML files
write_html(fig_ts, "docs/plots/time_series.html", config={'responsive': True})
write_html(fig_change, "docs/plots/annual_change.html")
write_html(fig_map, "docs/plots/choropleth.html")

# Save processed data
aespi.to_csv("data/processed/aespi_clean.csv", index=False)
regions.to_file("data/processed/canada_regions.geojson", driver='GeoJSON')

# %% [markdown]
# ## 5. Verification

# %%
# Check file existence
from IPython.display import FileLinks
FileLinks("docs/plots")

# Validation
assert Path("docs/plots/time_series.html").exists()
assert Path("data/processed/aespi_clean.csv").exists()
print("✅ All files exported successfully!")